In [1]:
%load_ext autoreload
%autoreload 2

# Feature Pipeline: Computing features

In [2]:
import os
import sys

repo_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_path)

import torch
import warnings
import polars as pl

from pprint import pprint
from loguru import logger
from sentence_transformers import SentenceTransformer

# Config
from recsys.config import settings

# Feature Store
# from recsys import gcp_integrations
from recsys.gcp_integrations import feature_store, bq_utils

# Features
from recsys.core.features.article_features import compute_features_articles, generate_embeddings_for_dataframe
from recsys.core.features.ranking_features import compute_rankings_dataset
from recsys.core.features.customer_features import DatasetSampler, compute_features_customers
from recsys.core.features.interaction_features import generate_interaction_data
from recsys.core.features.transaction_features import compute_features_transactions

# Raw Data
from recsys.raw_data_sources import h_and_m_raw_data

warnings.filterwarnings("ignore")

/opt/homebrew/Caskroom/miniconda/base/envs/mlops/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ONLINE: bool = False

# 🗄️ Articles data

The **article_id** and **product_code** serve different purposes in the context of H&M's product database:

- **Article ID**: This is a unique identifier assigned to each individual article within the database. It is typically used for internal tracking and management purposes. Each distinct item or variant of a product (e.g., different sizes or colors) would have its own unique article_id.

- **Product Code**: This is also a unique identifier, but it is associated with a specific product or style rather than individual articles. It represents a broader category or type of product within H&M's inventory. Multiple articles may share the same product code if they belong to the same product line or style.

While both are unique identifiers, the article_id is specific to individual items, whereas the product_code represents a broader category or style of product.

Here is an example:

**Product: Basic T-Shirt**

- **Product Code:** TS001

- **Article IDs:**
    - Article ID: 1001 (Size: Small, Color: White)
    - Article ID: 1002 (Size: Medium, Color: White)
    - Article ID: 1003 (Size: Large, Color: White)
    - Article ID: 1004 (Size: Small, Color: Black)
    - Article ID: 1005 (Size: Medium, Color: Black)

In this example, "TS001" is the product code for the basic t-shirt style. Each variant of this t-shirt (e.g., different sizes and colors) has its own unique article_id.



In [4]:
if ONLINE:
    articles_df = h_and_m_raw_data.extract_articles_df()
else:
    articles_df = pl.read_csv(source=f"{repo_path}/data/articles.csv")

articles_df.shape

(105542, 25)

In [5]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
i64,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,str
108775015,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775044,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775051,108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"


## Articles feature engineering

In [6]:
articles_df = compute_features_articles(articles_df, ONLINE, repo_path)
articles_df.shape

(105542, 27)

In [7]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,article_description,image_url
str,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,u32,str,str
"""108775015""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""/Users/galcala/Desktop/Github/…"
"""108775044""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""/Users/galcala/Desktop/Github/…"
"""108775051""",108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",13,"""Strap top (1) - Vest top in Ga…","""/Users/galcala/Desktop/Github/…"


## Create embeddings from the articles description

In [8]:
for i, desc in enumerate(articles_df["article_description"].head(n=3)):
    logger.info(f"Item {i + 1}:\n{desc}")

2025-02-12 14:46:41.085 | INFO     | __main__:<module>:2 - Item 1:
Strap top - Vest top in Garment Upper body
 Apperance: Solid
 Color: Dark Black 9
 Category: Ladieswear Womens Everyday Basics Jersey Basic
 Details: Jersey top with narrow shoulder straps.
2025-02-12 14:46:41.086 | INFO     | __main__:<module>:2 - Item 2:
Strap top - Vest top in Garment Upper body
 Apperance: Solid
 Color: Light White 10
 Category: Ladieswear Womens Everyday Basics Jersey Basic
 Details: Jersey top with narrow shoulder straps.
2025-02-12 14:46:41.086 | INFO     | __main__:<module>:2 - Item 3:
Strap top (1) - Vest top in Garment Upper body
 Apperance: Stripe
 Color: Dusty Light White 11
 Category: Ladieswear Womens Everyday Basics Jersey Basic
 Details: Jersey top with narrow shoulder straps.


In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

logger.info(
    f"Loading ${settings.FEATURES_EMBEDDING_MODEL_ID} embedding model to {device=}"
)

# Load embedding model from SentenceTransformers model registry
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

2025-02-12 14:46:41.142 | INFO     | __main__:<module>:9 - Loading $all-MiniLM-L6-v2 embedding model to device='mps'


In [10]:
articles_df = generate_embeddings_for_dataframe(
    articles_df, "article_description", model, batch_size=128
)

Generating embeddings...: 100%|██████████| 105542/105542 [05:40<00:00, 310.14it/s]


For each article description, we have a numerical vector which we can feed to a model, opposite to a string containing the description of an object.

In [11]:
articles_df[["article_description", "embeddings"]].head(3)

article_description,embeddings
str,list[f64]
"""Strap top - Vest top in Garmen…","[-0.023064, 0.069894, … 0.012397]"
"""Strap top - Vest top in Garmen…","[-0.011304, 0.075073, … 0.016485]"
"""Strap top (1) - Vest top in Ga…","[-0.033244, 0.081816, … 0.020111]"


In [12]:
articles_df["image_url"][0]

'/Users/galcala/Desktop/Github/GenAI_Custom_Real_Time_Personalized_Recommender/data/images/010/0108775015.jpg'

In [13]:
from IPython.display import HTML, display

image_urls = articles_df["image_url"].tail(12).to_list()
grid_html = '<div style="display: grid; grid-template-columns: repeat(6, 1fr); gap: 10pxl max-width: 900px;">'

for url in image_urls:
    grid_html += f'<img src="{url}" style="width: 100%; height: auto;">'

grid_html += "</div>"

display(HTML(grid_html))

# 👯‍♀️ Customers Data

In [14]:
if ONLINE:
    customers_df = h_and_m_raw_data.extract_customers_df()
else:
    customers_df = pl.read_csv(source=f"{repo_path}/data/customers.csv")

customers_df.shape

(1371980, 7)

In [15]:
customers_df.head(3)

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"


In [16]:
customers_df.null_count()

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
u32,u32,u32,u32,u32,u32,u32
0,895050,907576,6062,16009,15861,0


## Customers feature engineering

In [17]:
customers_df.head(3)

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"


In [18]:
customers_df = compute_features_customers(customers_df, drop_null_age=True)
customers_df.shape

(1356119, 5)

In [19]:
customers_df.head(3)

customer_id,club_member_status,age,postal_code,age_group
str,str,f64,str,str
"""00000dbacae5abe5e23885899a1fa4…","""ACTIVE""",49.0,"""52043ee2162cf5aa7ee79974281641…","""46-55"""
"""0000423b00ade91418cceaf3b26c6a…","""ACTIVE""",25.0,"""2973abc54daa8a5f8ccfe9362140c6…","""19-25"""
"""000058a12d5b43e67d225668fa1f8d…","""ACTIVE""",24.0,"""64f17e6a330a85798e4998f62d0930…","""19-25"""


# 🧾 Transactions Data

In [58]:
if ONLINE:
    transactions_df = h_and_m_raw_data.extract_transactions_df()
else:
    transactions_df = pl.read_csv(source=f"{repo_path}/data/transactions_train.csv")

transactions_df.shape

(31788324, 5)

In [59]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id
str,str,i64,f64,i64
"""2018-09-20""","""000058a12d5b43e67d225668fa1f8d…",663713001,0.050831,2
"""2018-09-20""","""000058a12d5b43e67d225668fa1f8d…",541518023,0.030492,2
"""2018-09-20""","""00007d2de826758b65a93dd24ce629…",505221004,0.015237,2


## Transactions feature engineering

In [60]:
transactions_df["t_dat"].head(3)

t_dat
str
"""2018-09-20"""
"""2018-09-20"""
"""2018-09-20"""


In [61]:
transactions_df = compute_features_transactions(transactions_df)
transactions_df.shape

(31788324, 9)

The time of the year a purchase was made should be a strong predictor, as seasonality plays a big factor in fashion purchases. Here, you will use the month of the purchase as a feature. Since this is a cyclical feature (January is as close to December as it is to February), you'll map each month to the unit circle using sine and cosine.

Thus, the features of the transactions DataFrame look as follows:

In [62]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week
i64,str,str,f64,i64,i32,i8,i8,i8
1537401600,"""000058a12d5b43e67d225668fa1f8d…","""663713001""",0.050831,2,2018,9,20,4
1537401600,"""000058a12d5b43e67d225668fa1f8d…","""541518023""",0.030492,2,2018,9,20,4
1537401600,"""00007d2de826758b65a93dd24ce629…","""505221004""",0.015237,2,2018,9,20,4


We don't want to work with ~30 million transactions in these series, as everything will take too much time to run. Thus, we create a subset of the original dataset by randomly sampling from the customers' datasets and taking only their transactions.

In [63]:
sampler = DatasetSampler(size=settings.CUSTOMER_DATA_SIZE)

dataset_subset = sampler.sample(
    customers_df=customers_df, transactions_df=transactions_df
)

customers_df = dataset_subset["customers"]
transactions_df = dataset_subset["transactions"]

2025-02-13 14:44:55.183 | INFO     | recsys.features.customers:sample:29 - Sampling 1000 customers.
2025-02-13 14:44:55.187 | INFO     | recsys.features.customers:sample:32 - Number of transactions for all the customers: 31788324
2025-02-13 14:44:56.793 | INFO     | recsys.features.customers:sample:38 - Number of transactions for the 1000 sampled customers: 23799


In [64]:
transactions_df.shape

(23799, 9)

Some of the remaining customers:

In [65]:
for customer_id in transactions_df["customer_id"].unique().head(10):
    logger.info(f"Logging customer ID: {customer_id}")

2025-02-13 14:44:59.965 | INFO     | __main__:<module>:2 - Logging customer ID: 127401ef26829611b47d7d3054081e3bd55cf3736713c190e17b785ce5fa3ea2
2025-02-13 14:44:59.966 | INFO     | __main__:<module>:2 - Logging customer ID: aec5915f7acec57feee7f5e2859d7380a37e7134e77e9cb7e0cd88cb8928fe99
2025-02-13 14:44:59.966 | INFO     | __main__:<module>:2 - Logging customer ID: 71d2013d4b6f305fd6d37aafd004863dc488d69bec46fb63d6cbce8f29b87f11
2025-02-13 14:44:59.966 | INFO     | __main__:<module>:2 - Logging customer ID: a68809465a403f0f1358141f3f72889ce5a496d7d2b49b66defcae454f5c108b
2025-02-13 14:44:59.966 | INFO     | __main__:<module>:2 - Logging customer ID: 76b4d85c5909dff1f26354958ec58d77ba222330f7ccd42688a2a0848e808369
2025-02-13 14:44:59.966 | INFO     | __main__:<module>:2 - Logging customer ID: 87e9fd5dbe5bb960cbb8f86e7cf535de45008a9ee02df382b31ef12ba626a0e3
2025-02-13 14:44:59.967 | INFO     | __main__:<module>:2 - Logging customer ID: b7e0003e10d72ce5e562d4fc2b09f18ab1cd375aad94372110

# 🤳🏻 Interaction data

To train our models, we need more than just the transactions DataFrame. We need positive samples that signal whether a customer clicked or bought an item, but we also need negative samples that signal no interactions between a customer and an item.

In [28]:
interaction_df = generate_interaction_data(transactions_df)
interaction_df.shape

Processing customer chunks:   0%|          | 0/1 [00:03<?, ?it/s]


(135123, 5)

In [29]:
interaction_df.head()

t_dat,customer_id,article_id,interaction_score,prev_article_id
i64,str,str,i64,str
1261454400,"""00b203a32faa3d007dba198ef27c15…","""775328004""",0,"""START"""
1267574400,"""00b203a32faa3d007dba198ef27c15…","""710729005""",0,"""775328004"""
1280534400,"""00b203a32faa3d007dba198ef27c15…","""689009001""",0,"""710729005"""
1283414400,"""00b203a32faa3d007dba198ef27c15…","""838780003""",0,"""689009001"""
1284854400,"""00b203a32faa3d007dba198ef27c15…","""792517002""",0,"""838780003"""


Score distributions:

Here is what each score means:
- `0` : No interaction between a customer and an item
- `1` : A customer clicked an item
- `2` : A customer bought an item

In [30]:
interaction_df.group_by("interaction_score").agg(
    pl.count("interaction_score").alias("total_interactions")
)

interaction_score,total_interactions
i64,u32
0,73152
1,38172
2,23799


# Upload feature group

### Customers

In [31]:
customers_df.head()

customer_id,club_member_status,age,postal_code,age_group
str,str,f64,str,str
"""6f173c0d871007b835f77d16232bae…","""ACTIVE""",61.0,"""aacae4bc0b6cf9c6708b05f234512a…","""56-65"""
"""1ae5ddce4cdf0f21ae41379a6edad9…","""ACTIVE""",23.0,"""2c29ae653a9282cce4151bd87643c9…","""19-25"""
"""b903a2e7b1fba1924a2332c1b69fc4…","""ACTIVE""",25.0,"""612ae24e8d0ea791da6b5b4de91714…","""19-25"""
"""f17d07ee3b52dc06ba23e5dbd0621a…","""ACTIVE""",33.0,"""caaf654c6a82da724a42162a80f037…","""26-35"""
"""27e13eb33e457d98f0f7fd086231a4…","""ACTIVE""",35.0,"""f919da058a4a967dd830c3cbb3c657…","""26-35"""


In [32]:
logger.info("Uploading 'customers' Feature to BigQuery.")
bq_utils.load_features_to_bigquery(customers_df=customers_df)
logger.info("✅ Uploaded 'customers' Feature to BigQuery!")

2025-02-12 14:52:35.530 | INFO     | __main__:<module>:1 - Uploading 'customers' Feature to BigQuery.
2025-02-12 14:52:35.566 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted customer_id to STRING
2025-02-12 14:52:35.566 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted club_member_status to STRING
2025-02-12 14:52:35.567 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted age to FLOAT
2025-02-12 14:52:35.567 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted postal_code to STRING
2025-02-12 14:52:35.567 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted age_group to STRING
2025-02-12 14:52:35.567 | DEBUG    | recsys.gcp_integrations.bq_utils:upload_dataframe_to_bigquery:74 - DataFrame types before upload:
2025-02-12 14:52:35.568 | DEBUG    | recsys.gcp_integrations.bq_utils:upload_dataframe_to_bigquer

### Articles

In [33]:
logger.info("Uploading 'articles' Feature to BigQuery.")
bq_utils.load_features_to_bigquery(articles_df=articles_df)
logger.info("✅ Uploaded 'articles' Feature to BigQuery!")

2025-02-12 14:52:40.885 | INFO     | __main__:<module>:1 - Uploading 'articles' Feature to BigQuery.
2025-02-12 14:52:41.112 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted article_id to STRING
2025-02-12 14:52:41.114 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted prod_name to STRING
2025-02-12 14:52:41.115 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted prod_name_length to INTEGER
2025-02-12 14:52:41.116 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted product_type_name to STRING
2025-02-12 14:52:41.118 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted product_group_name to STRING
2025-02-12 14:52:41.119 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted graphical_appearance_name to STRING
2025-02-12 14:52:41.120 | DEBUG    | recsys.gcp_integrations.bq_u

### Transactions

In [66]:
transactions_df.head()

t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week
i64,str,str,f64,i64,i32,i8,i8,i8
1537401600,"""25f0bcb3b9e6fffee7541b02d21e48…","""541518004""",0.030492,1,2018,9,20,4
1537401600,"""25f0bcb3b9e6fffee7541b02d21e48…","""654564002""",0.016932,1,2018,9,20,4
1537401600,"""25f0bcb3b9e6fffee7541b02d21e48…","""372860002""",0.013542,1,2018,9,20,4
1537401600,"""4e82363f3c5a710922073cdf626309…","""673285001""",0.025407,1,2018,9,20,4
1537401600,"""4e82363f3c5a710922073cdf626309…","""564314018""",0.025407,1,2018,9,20,4


In [67]:
logger.info("Uploading 'transactions' Feature to BigQuery.")
bq_utils.load_features_to_bigquery(transactions_df=transactions_df)
logger.info("✅ Uploaded 'transactions' Feature to BigQuery!")

2025-02-13 14:45:13.422 | INFO     | __main__:<module>:1 - Uploading 'transactions' Feature to BigQuery.
2025-02-13 14:45:13.477 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:46 - Converted t_dat to INTEGER
2025-02-13 14:45:13.478 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:46 - Converted customer_id to STRING
2025-02-13 14:45:13.478 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:46 - Converted article_id to STRING
2025-02-13 14:45:13.479 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:46 - Converted price to FLOAT
2025-02-13 14:45:13.479 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:46 - Converted sales_channel_id to INTEGER
2025-02-13 14:45:13.479 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:46 - Converted year to INTEGER
2025-02-13 14:45:13.480 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:46 - Co

### Interactions

In [36]:
interaction_df.head(3)

t_dat,customer_id,article_id,interaction_score,prev_article_id
i64,str,str,i64,str
1261454400,"""00b203a32faa3d007dba198ef27c15…","""775328004""",0,"""START"""
1267574400,"""00b203a32faa3d007dba198ef27c15…","""710729005""",0,"""775328004"""
1280534400,"""00b203a32faa3d007dba198ef27c15…","""689009001""",0,"""710729005"""


In [37]:
logger.info("Uploading 'interactions' Feature to BigQuery.")
bq_utils.load_features_to_bigquery(interactions_df=interaction_df)
logger.info("✅ Uploaded 'interactions' Feature to BigQuery!")

2025-02-12 14:55:05.800 | INFO     | __main__:<module>:1 - Uploading 'interactions' Feature to BigQuery.
2025-02-12 14:55:06.234 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted t_dat to INTEGER
2025-02-12 14:55:06.237 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted customer_id to STRING
2025-02-12 14:55:06.238 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted article_id to STRING
2025-02-12 14:55:06.239 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted interaction_score to INTEGER
2025-02-12 14:55:06.241 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted prev_article_id to STRING
2025-02-12 14:55:06.241 | DEBUG    | recsys.gcp_integrations.bq_utils:upload_dataframe_to_bigquery:74 - DataFrame types before upload:
2025-02-12 14:55:06.242 | DEBUG    | recsys.gcp_integrations.bq_utils:upload_datafram

# Compute ranking dataset

## TO-DO: REPLACE THE DFS WITH THE DATA FROM VERTEX AI FEATURE ONLINE STORE AS WE WANT TO REFLECT THE LINEAGE OF OUR SOURCE OF TRUTH

The last step is to compute the ranking dataset used to train the scoring/ranking model from the feature groups we've just created:


In [38]:
fos = feature_store.get_feature_store()

2025-02-12 14:55:12.230 | INFO     | recsys.gcp_integrations.feature_store:get_feature_store:21 - Retrieving Feature Store from us-central1/recsys-dev-gonzo/recsys_feature_store_dev
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [43]:
trans_fv, articles_fv, customers_fv = feature_store.create_retrieval_feature_view(fos)

In [48]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

ranking_df = compute_rankings_dataset(trans_fv, articles_fv, customers_fv)

2025-02-12 15:24:58.286 | INFO     | recsys.features.ranking:compute_rankings_dataset:48 - Starting to compute rankings dataset
2025-02-12 15:24:58.286 | INFO     | recsys.features.ranking:compute_rankings_dataset:51 - Fetching transactions data...
2025-02-12 15:24:58.286 | INFO     | recsys.features.ranking:fetch_feature_view_data:17 - Starting to fetch data from feature view: transactions
2025-02-12 15:24:58.910 | INFO     | recsys.features.ranking:fetch_feature_view_data:31 - Executing query: SELECT article_id, customer_id FROM `recsys-dev-gonzo.recsys_dataset.recsys_transactions`
2025-02-12 15:25:02.071 | INFO     | recsys.features.ranking:fetch_feature_view_data:35 - DataFrame shape: (23799, 2)
2025-02-12 15:25:02.072 | INFO     | recsys.features.ranking:compute_rankings_dataset:58 - Fetching articles data...
2025-02-12 15:25:02.072 | INFO     | recsys.features.ranking:fetch_feature_view_data:17 - Starting to fetch data from feature view: articles
2025-02-12 15:25:02.559 | INFO   

In [49]:
ranking_df.head(3)

customer_id,article_id,age,label,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length
str,str,f64,i32,i64,str,i64,str,str,i64,str,str,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,i64
"""d2a82f9666063659c9bbb393a01616…","""720101003""",43.0,1,720101,"""NORMA RW cropped conscious""",272,"""Trousers""","""Garment Lower body""",1010016,"""Solid""","""93""","""Dark Green""",4,"""Dark""",19,"""Green""",8397,"""Ladies Sport Woven""","""S""","""Sport""",26,"""Sport""",5,"""Ladies H&M Sport""",1001,"""Unknown""",26
"""876feebe6fef4c8ea7493b54b3fa47…","""694410001""",37.0,1,694410,"""Kendal sandalette""",92,"""Heeled sandals""","""Shoes""",1010016,"""Solid""","""9""","""Black""",4,"""Dark""",5,"""Black""",3528,"""Heels""","""C""","""Ladies Accessories""",1,"""Ladieswear""",64,"""Womens Shoes""",1020,"""Shoes""",17
"""66462ecf13fd33a15dc49313f5a0e2…","""616920001""",26.0,1,616920,"""Liza Strapless Sofia 2pk""",306,"""Bra""","""Underwear""",1010016,"""Solid""","""12""","""Light Beige""",1,"""Dusty Light""",11,"""Beige""",1334,"""Casual Lingerie""","""B""","""Lingeries/Tights""",1,"""Ladieswear""",61,"""Womens Lingerie""",1017,"""Under-, Nightwear""",24


In [50]:
ranking_df.get_column("label").value_counts()

label,count
i32,u32
1,20376
0,203760


As the ranking dataset was computed based on articles, customers, and transactions Feature Views, we can reflect this lineage in the ranking Feature View.

In [51]:
logger.info("Uploading 'rankings' Feature to BigQuery.")
bq_utils.load_features_to_bigquery(rankings_df=ranking_df)
logger.info("✅ Uploaded 'rankings' Feature to BigQuery!")

2025-02-12 15:26:30.129 | INFO     | __main__:<module>:1 - Uploading 'rankings' Feature to BigQuery.
2025-02-12 15:26:30.281 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted customer_id to STRING
2025-02-12 15:26:30.284 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted article_id to STRING
2025-02-12 15:26:30.284 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted age to FLOAT64
2025-02-12 15:26:30.285 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted label to INTEGER
2025-02-12 15:26:30.287 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted product_type_name to STRING
2025-02-12 15:26:30.289 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquery:45 - Converted product_group_name to STRING
2025-02-12 15:26:30.292 | DEBUG    | recsys.gcp_integrations.bq_utils:convert_types_for_bigquer